# <span style="color:darkblue"> Lecture 12 - Replacing and recoding variables</span>

<font size = "5">

- Starting a new module on **manipulating data**
- We will discuss NaNs and how to clean data

**Covered so far**

1. Python types and controlling flows (if/elif/else)

2. Local vs global variables

3. Writing and running functions and lambda functions and the apply method

4. Subsetting data using queries

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">
Key libraries

In [1]:
import numpy as np
import pandas as pd

<font size = "5">

Read dataset on car racing circuits

- https://en.wikipedia.org/wiki/Formula_One <br>
- [See Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [2]:
circuits = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

The dataset "codebook" is a table with ...

- Key column information
- Main things:  Field, Type, and Description

<img src="figures/codebook_circuits.png" alt="drawing" width="500"/>

- Codebook^^ - gives necesarry information about the data you are going to use 

In [ ]:
# The codebook contains basics about the columns

# "Field" is the name given to the name of the column

# "Type"  is the variable type (with the number of characters in parenthesis - max number of characters):
#         integer (int)
#         string (varchar - "variable character") 
#         float (float)

# "Description" contains a label with the content of the variable

<font size = "5">

Quick Discussion:
- What does the "alt" column represent?

- "alt" represents altitude (in meters), which in this case is an integer 

- What does "varchar(255)" mean?

- "varchar(255)" , 255 represnts the maximum number of characters that the string can contain , the "var" part means that it is a variable 

# <span style="color:darkblue"> II. NaNs </span>

<font size = "5">

- Means "Not a Number"
- Used to denote missing values

In [3]:
# "NaNs" are a special number, available in the "np" library

np.nan

nan

<font size = "5">

Operations on arrays with NaNs

In [5]:
# Create two array with and without "NaNs"
# The "np.array()" functions converts a list to an array

vec_without_nans = np.array([1,1,1])
vec_with_nans = np.array([np.nan,4,5])

# When you add the vectors
# you will produce a NaN on any entries with "NaNs"
print(vec_without_nans * vec_with_nans)
print(vec_without_nans / vec_with_nans)
print(vec_without_nans + vec_with_nans)
print(vec_without_nans - vec_with_nans)

# Any position in which you have a missing value, you will end up with a result of a missing number "nan"

[nan  4.  5.]
[ nan 0.25 0.2 ]
[nan  5.  6.]
[nan -3. -4.]


<font size = "5">

Summary statistics with NaNs

In [7]:
# Some summary statistics will not work if there are "NaNs"
# The "np.mean()" doesn't work if the mean includes "NaNs", will not work even if there is one "nan"
print(np.mean(vec_with_nans))

# Some commands ignore the "nans"
# The "np.nanmean()" computes the mean over the numeric obvservations, will ignore the "nan" value 
print(np.nanmean(vec_with_nans))

nan
4.5


<font size = "5">

Pandas summary statistics ignore NaNs

In [11]:
# This command creates an empty dataframe
# then creates a new column called "x" and computes its mean
# Note: If a column contains missing values, then the "mean" won't be 
#       representative of the whole sample

dataset = pd.DataFrame()
dataset["x"] = vec_with_nans
print(dataset)
dataset["x"].mean()

     x
0  NaN
1  4.0
2  5.0


4.5

# <span style="color:darkblue"> II. Data Cleaning</span>

<font size = "5">

- Data collection isn't perfect!
- Need to adjust values with incorrect formatting

<font size = "5">
Get data types

In [13]:
# Produces a list with the data types of each column
# Columns that say "object" have either strings or 
# a mix of string and numeric values

circuits.dtypes

# We know from the codebook that "alt" should be a number, but because the ^^ command returns object, we jnow that it is a mix of numeric values and strings

circuitId       int64
circuitRef     object
name           object
location       object
country        object
lat           float64
lng           float64
alt            object
url            object
dtype: object

<font size = "5">

Check rows with numeric values

In [15]:
# The ".str.isnumeric()" checks whether each row is numeric or not.
# Using the "." twice is an example of "piping"
# which refers to chaining two commands "str" and "isnumeric()"

circuits["alt"].str.isnumeric()

0      True
1      True
2      True
3      True
4      True
      ...  
72     True
73     True
74     True
75    False
76    False
Name: alt, Length: 77, dtype: bool

<font size = "5">

Extract list of non-numeric values

In [20]:
circuits.query("alt.str.isnumeric() == False")

# If "alt" is a global variable then use "@alt"

# You get 4 rows 

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
22,80,vegas,Las Vegas Strip Street Circuit,Las Vegas,United States,36.1147,-115.1730,\N,https://en.wikipedia.org/wiki/Las_Vegas_Grand_...
71,73,baku,Baku City Circuit,Baku,Azerbaijan,40.3725,49.8533,-7,http://en.wikipedia.org/wiki/Baku_City_Circuit
75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.4900,51.4542,\N,http://en.wikipedia.org/wiki/Losail_Internatio...
76,79,miami,Miami International Autodrome,Miami,USA,25.9581,-80.2389,\N,http://en.wikipedia.org/wiki/Miami_Internation...


In [19]:
# We can reference subattributes of columns in ".query()"
# The pd.unique() function extracts unique values from a list

subset = circuits.query("alt.str.isnumeric() == False")

# Creating a subset of all the rows for a variable is not numeric 

list_unique = pd.unique(subset["alt"])
print(list_unique)


['\\N' '-7']


<font size = "5">

Replace certain values

In [21]:
# "list_old" encodes values we want to change
# "list_new" encodes the values that will "replace" the old
list_old = ['\\N','-7'] # -7 is categorized as a string so you have to put it in ""
list_new = [np.nan,-7] # replacing "\\N" with "np.nan"

# This command replaces the values of the "alt" column
circuits["alt"] = circuits["alt"].replace(list_old, list_new)
circuits["alt"]

0      10
1      18
2       7
3     109
4     130
     ... 
72    108
73    255
74     15
75    NaN
76    NaN
Name: alt, Length: 77, dtype: object

In [24]:
circuits["alt"] = pd.to_numeric(circuits["alt"])
# Transforms all elements to floats 
# Need to save it as an object as otherwise will not save 

In [25]:
circuits["alt"][0]

# Now it is a number 

10.0

<font size = "5">

Store a "cleaned" dataset

In [27]:
# After the cleaning process is done, you may want to store the dataset again
# It's recommended to do this in a separate file from the original
# That way you can always go back to the original if you made a coding error

circuits.to_csv("data_clean/circuits.csv")

# Saving cleaned dataset into a seperate file/folder 


<font size = "5">
Try it yourself!

- Use ".replace()" with the "country" column
- Replace "UK" with "United Kingdom"

In [ ]:
# Write your own code


<font size = "5">
Try it yourself!

- What is the column type of "lat" or "lng"?
- Does it have any string variables?
- Can we use ```str.isnumeric()``` here?

In [30]:
# Write your own code
circuits['lat'].str.isnumeric()

# Cannot use it because it can only be used with string datasets, not ones that are already a float 

AttributeError: Can only use .str accessor with string values!

<font size = "5">

# <span style="color:darkblue"> II. Recoding Numeric Variables </span>



<font size = "5">

<span style="color:red"> Controlled Pitfall </span> Computing a mean for a non-numeric column

In [31]:
circuits["alt"].dtype

dtype('float64')

In [ ]:
# Uncomment this command to see the error
# The following error occurs because the data type
# is not numeric
# circuits["alt"].mean()

<font size = "5">

Convert column to numeric

In [33]:
# pd.to_numeric() converts a column to numeric
# Before you use this option, make sure to "clean" the variable

# CLEAN BEFORE YOU CONVERT TO NUMERIC 

# as we did before by checking what the non-numeric values are

circuits["alt_numeric"] = pd.to_numeric(circuits["alt"])
print(circuits["alt_numeric"].mean())

248.1891891891892


<font size = "5">

Recode values based on an interval <br>

$ \qquad x_{bin} = \begin{cases} ``A" &\text{ if \quad} x_1 < x \le x_2 \\
                                  ``B" &\text{ if \quad} x_2 < x \le x_3 \end{cases} $



In [36]:
bins_x = [0,2500, 5000]
# Defining cutoffs that are in bins_x vector above ^^ - bins will be the set of thresholds that determine how you want to divide the data 
labels_x = ["Between 0 and 2500",
            "Between 2500 and 5000"]

circuits["bins_alt"] = pd.cut(circuits["alt_numeric"],
                              bins = bins_x,
                              right = True,
                              labels = labels_x)

# Note: if we set bins_x = [float("-inf"),2500, float("inf")]
#       then intervals are "Less than or equal to 2500" and "Above 2500"
#       float("inf") and float("-inf") represent infinity and negative infinity
#       The "right" command indicates that the right interval is
#       "less than or equal to" or just "less than"
circuits["bins_alt"]
circuits.query("alt_numeric < 2500")

,circuitId,circuitRef,name,location,country,lat,lng,alt,url,alt_numeric,bins_alt
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,10.0,Between 0 and 2500
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18.0,http://en.wikipedia.org/wiki/Sepang_Internatio...,18.0,Between 0 and 2500
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7.0,http://en.wikipedia.org/wiki/Bahrain_Internati...,7.0,Between 0 and 2500
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109.0,http://en.wikipedia.org/wiki/Circuit_de_Barcel...,109.0,Between 0 and 2500
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130.0,http://en.wikipedia.org/wiki/Istanbul_Park,130.0,Between 0 and 2500
...,...,...,...,...,...,...,...,...,...,...,...
70,71,sochi,Sochi Autodrom,Sochi,Russia,43.40570,39.95780,2.0,http://en.wikipedia.org/wiki/Sochi_Autodrom,2.0,Between 0 and 2500
71,73,baku,Baku City Circuit,Baku,Azerbaijan,40.37250,49.85330,-7.0,http://en.wikipedia.org/wiki/Baku_City_Circuit,-7.0,NaN
72,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108.0,http://en.wikipedia.org/wiki/Algarve_Internati...,108.0,Between 0 and 2500
73,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255.0,http://en.wikipedia.org/wiki/Mugello_Circuit,255.0,Between 0 and 2500


<font size = "5">
Try it yourself!

- Create a new variable "hemisphere"
- Encode lattitude in (-90 and 0] as "south"
- Encode lattitude in (0 and 90] as "north"

In [ ]:
# Write your own code
